In [10]:
import datetime
from pathlib import Path
import isodate

import sys
import os

gribscan_directory = f"{os.path.abspath('')}/../src/"
sys.path.append(gribscan_directory)

from gribscan_harmonie.load import create_loader

/dmidata/users/kah/git/data-catalog/.venv/lib/python3.10/site-packages/gribapi/__init__.py:23: UserWarning: ecCodes 2.31.0 or higher is recommended. You are running version 2.24.2
  warnings.warn(
/dmidata/users/kah/git/data-catalog/.venv/lib/python3.10/site-packages/gribscan/gridutils.py:192: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _calc_lambda_phi_numba(east, north, false_easting, false_northing, r0, theta_0, a, k0, F, n, e):


In [11]:
FILEPATH_ROOT = "/dmidata/cache/mdc{partition}/gdb/grib2/dini/"
FILEPATH_SUFFIX_FORMAT = "{data_kind}/{analysis_time:%Y%m%d%H}"
FILENAME_FORMAT = "{forecast_hour:03d}"
FORECAST_RESOLUTION = datetime.timedelta(hours=1)
FORECAST_DURATION = datetime.timedelta(hours=54)
FORECAST_PRODUCTION_START = isodate.parse_datetime("2024-03-19T00:00Z")


def find_dini_grib_files_collection(
    analysis_time: datetime.datetime, forecast_duration: datetime.timedelta = None
):
    assert isinstance(analysis_time, datetime.datetime)

    if analysis_time >= FORECAST_PRODUCTION_START:
        partition = "prd"
    else:
        partition = "dev"

    fp_forecast_root = Path(
        FILEPATH_ROOT.format(partition=partition)
    ) / FILEPATH_SUFFIX_FORMAT.format(analysis_time=analysis_time, data_kind="sf")

    if forecast_duration is None:
        forecast_duration = FORECAST_DURATION

    n_output_steps = forecast_duration // FORECAST_RESOLUTION

    fps_grib = [
        fp_forecast_root / FILENAME_FORMAT.format(forecast_hour=forecast_hour)
        for forecast_hour in range(n_output_steps)
    ]

    return fps_grib


find_dini_grib_files_collection.dt_collection_analysis_timespan = None

In [17]:
t_analysis = (datetime.datetime.utcnow() - datetime.timedelta(hours=6)).replace(
    minute=0, second=0, microsecond=0
)
t_analysis = t_analysis - datetime.timedelta(hours=t_analysis.hour % 6)

fp_grib_indecies_root = Path("/dmidata/projects/nwp/tmp/grib-indecies/")
fp_grib_indecies_root.mkdir(exist_ok=True, parents=True)

harmonie_loader = create_loader(
    fn_source_files=find_dini_grib_files_collection,
    fp_grib_indecies_root=fp_grib_indecies_root,
)

#ds = harmonie_loader(
#    t_analysis=slice(t_analysis - datetime.timedelta(hours=6), t_analysis, "PT3H"),
#    level_type="heightAboveGround",
#)

ds = harmonie_loader(
    t_analysis=t_analysis,
    level_type="heightAboveGround",
)

/dmidata/users/kah/git/gribscan-harmonie/notebooks/../src/gribscan_harmonie/utils.py:19: UserWarning: timestamp has no timezone, assuming system timezone (UTC)
  warnings.warn(
2024-04-25 18:07:54.922 | DEBUG    | gribscan_harmonie.load:_write_zarr_indexes_for_grib_files:41 - Opening the following GRIB files: /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/000, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/001, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/002, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/003, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/004, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/005, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/006, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/007, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/008, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/009, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/010, /dmidata/cache/mdcprd/gdb/grib2/dini/sf/2024042512/011, /dmidata/cache/md

In [18]:
zarr_root = Path("/dmidata/projects/nwp/tmp/zarr/")
ds.to_zarr(f'{zarr_root}/test.zarr', mode="w")

<xarray.Dataset>
Dimensions:  (time: 54, y: 1606, x: 1906, level: 7)
Coordinates:
    lat      (y, x) float64 dask.array<chunksize=(1606, 1906), meta=np.ndarray>
  * level    (level) int64 0 2 50 100 150 250 300
    lon      (y, x) float64 dask.array<chunksize=(1606, 1906), meta=np.ndarray>
  * time     (time) datetime64[ns] 2024-04-25T12:00:00 ... 2024-04-27T17:00:00
  * x        (x) float64 -1.528e+06 -1.526e+06 -1.524e+06 ... 2.28e+06 2.282e+06
  * y        (y) float64 -1.589e+06 -1.587e+06 ... 1.619e+06 1.621e+06
Data variables: (12/40)
    100u     (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    100v     (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    10efg    (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    10nfg    (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    10si     (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    10u      (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    ...       ...
    ulwrf    (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    unknown  (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    v        (time, level, y, x) float64 dask.array<chunksize=(1, 1, 1606, 1906), meta=np.ndarray>
    vflx     (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    vis      (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
    z        (time, y, x) float64 dask.array<chunksize=(1, 1606, 1906), meta=np.ndarray>
Attributes:
    edition:            2
    centre:             ekmi
    centreDescription:  Copenhagen
    subCentre:          255
    history:            🪄🧙‍♂️🔮 magic dataset assembly provided by gribscan.Ha...